# Assignment 2 in DLWS - elo competition category Embedding

In this part of the assignmet we are trying to use embedding in  the elo kaggle competition which can be seen here : https://www.kaggle.com/c/elo-merchant-category-recommendation

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


Imports we use in the notebook

In [0]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from keras.utils import to_categorical
from keras.optimizers import Adam, SGD
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau, Callback
from keras.models import Model
from keras.layers import Input, Embedding , concatenate, Flatten, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
 

import gc


Using TensorFlow backend.


Path to load the data and loading hte train and test data

In [0]:
path = 'drive/My Drive/MY DLWS/Elo World/'

train = pd.read_csv(path + 'train.csv', parse_dates=['first_active_month'])
test = pd.read_csv(path + 'test.csv', parse_dates=['first_active_month'])
his_trans = pd.read_csv(path + 'historical_transactions.csv')


Taking only the categorical data from the history and train data and merge them to one dataframe

In [0]:
cat_train = train[['card_id','feature_1','feature_2','feature_3']]

cat_hist_trans = his_trans[['card_id','state_id']]

cat_train = pd.merge(cat_train, cat_hist_trans, how='left', on=['card_id'])

cat_train = cat_train.drop(['card_id'], axis=1)


In [0]:
## removing dataframes to save space - in this data, space is a real concern.
del his_trans
gc.collect()

cat_train.head()

,feature_1,feature_2,feature_3,state_id
0,5,2,1,9
1,5,2,1,9
2,5,2,1,9
3,5,2,1,9
4,5,2,1,9


Using category embedding like we saw in class - we try yo create an embedding between the 3 features in the train data to the state_id feature in the history and new transacrition data for start. the same code can be used again and again to create different embeddings from the data, or to create embedding from more features if we would like.

In [0]:
n_ftr_1 = cat_train['feature_1'].nunique()
n_ftr_2 = cat_train['feature_2'].nunique()
n_ftr_3 = cat_train['feature_3'].nunique()

n_state_id = cat_train['state_id'].nunique()

ftr_1 = {p:i for (i,p) in enumerate(cat_train['feature_1'].unique())}
ftr_2 = {p:i for (i,p) in enumerate(cat_train['feature_2'].unique())}
ftr_3 = {p:i for (i,p) in enumerate(cat_train['feature_3'].unique())}
             
state_id = {p:i for (i,p) in enumerate(cat_train['state_id'].unique())}





In [0]:
n_emb_output = 5

processed_his = cat_train.loc[:,['feature_1','feature_2','feature_3']].copy()


processed_his['feature_1'] = [ftr_1[x] for x in cat_train['feature_1']]
processed_his['feature_2'] = [ftr_2[x] for x in cat_train['feature_2']]
processed_his['feature_3'] = [ftr_3[x] for x in cat_train['feature_3']]


ftr_1_in = Input(shape=(1,), dtype='int64', name='ftr_1')
ftr_2_in = Input(shape=(1,), dtype='int64', name='ftr_2')
ftr_3_in = Input(shape=(1,), dtype='int64', name='ftr_3')

ftr_1_emb = Embedding(n_ftr_1, n_emb_output, input_length=1 ,embeddings_regularizer=l2(1e-4))(ftr_1_in)
ftr_2_emb = Embedding(n_ftr_2, n_emb_output, input_length=1 ,embeddings_regularizer=l2(1e-4))(ftr_2_in)
ftr_3_emb = Embedding(n_ftr_3, n_emb_output, input_length=1 ,embeddings_regularizer=l2(1e-4))(ftr_3_in)

In [0]:
x = concatenate([ftr_1_emb, ftr_2_emb, ftr_3_emb], name = 'concatenate')
x = Flatten()(x)
x = Dense(256, activation='relu', kernel_initializer='he_normal')(x)
x = Dense(128, activation='relu', kernel_initializer='he_normal')(x)
x = Dense(64, activation='relu', kernel_initializer='he_normal')(x)

x = Dense(n_state_id, activation='softmax')(x)


nn = Model([ftr_1_in, ftr_2_in, ftr_3_in], x)


target = to_categorical([state_id[x] for x in cat_train['state_id']])

nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
nn.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ftr_1 (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
ftr_2 (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
ftr_3 (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 5)         25          ftr_1[0][0]                      
__________________________________________________________________________________________________
embedding_

The idea of embedding as we understood it is to create new features for the data using the original categorical features we have in the train data and the categorical feautres in the other datasets. i.e we want to create a new fetures representing the connection between the original features and other feautres in the complete data and use this embedding to help improve the model results on the target. 

In [0]:
history = nn.fit([processed_his[f] for f in processed_his.columns], target, batch_size=64, validation_split=0.2, epochs=1)

Train on 14424007 samples, validate on 3606002 samples
Epoch 1/1
14424007/14424007 [==============================] - 2648s 184us/step - loss: 2.3810 - acc: 0.3643 - val_loss: 2.3838 - val_acc: 0.3623


In [0]:
nn.save_weights(path + 'my_model_weights.h5')

In [0]:
nn.load_weights(path + 'my_model_weights.h5')

**Using the embedding layer for loyalty predictions**


Here we used only the embedding we created to prdict the loyalty. the resutls are no so good as we can see. that might be beacuse the embedding wasnt done right or beacuse there is no real connection between the features so the knowledge gain in the embedding is no use.
also thete inght be there are not engouh features used to make a good predictaion.

In [0]:
main_model = Model(input = nn.input, output = nn.get_layer('concatenate').output)

for layer in main_model.layers:
   layer.trainable = False

y = main_model.output
y = Flatten()(y)
y = Dense(256, activation='relu', kernel_initializer='he_normal')(y)
y = BatchNormalization()(y)
y = Dense(128, activation='relu', kernel_initializer='he_normal')(y)
y = BatchNormalization()(y)
y = Dense(64, activation='relu', kernel_initializer='he_normal' )(y)
y = BatchNormalization()(y)
pred = Dense(1, activation='relu', kernel_initializer='he_normal')(y)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=2, min_lr=0.001)

class PrintLearningRate(Callback):
  def on_epoch_end(self, epoch, logs=None):
    print(K.eval(self.model.optimizer.lr))

print_lr = PrintLearningRate()

    
main_model = Model([ftr_1_in, ftr_2_in, ftr_3_in], pred)
sgd = SGD(lr=0.1, momentum=0.9, nesterov=True)


    
    
main_model.compile(optimizer=sgd, loss='mse')
main_model.summary()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("co...)`
  """Entry point for launching an IPython kernel.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ftr_1 (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
ftr_2 (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
ftr_3 (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 5)         25          ftr_1[0][0]                      
__________________________________________________________________________________________________
embedding_

In [0]:
processed_train = train.loc[:,['feature_1','feature_2','feature_3']].copy()
processed_train['feature_1'] = [ftr_1[x] for x in train['feature_1']]
processed_train['feature_2'] = [ftr_2[x] for x in train['feature_2']]
processed_train['feature_3'] = [ftr_3[x] for x in train['feature_3']]


history = main_model.fit([processed_train[f] for f in processed_train.columns], train.target, validation_split=0.2, batch_size=128, epochs=10, callbacks=[reduce_lr, print_lr])



Train on 161533 samples, validate on 40384 samples
Epoch 1/10
161533/161533 [==============================] - 15s 95us/step - loss: 14.8773 - val_loss: 15.5280
0.1
Epoch 2/10
161533/161533 [==============================] - 15s 90us/step - loss: 14.8446 - val_loss: 15.5280
0.1
Epoch 3/10
161533/161533 [==============================] - 15s 91us/step - loss: 14.8446 - val_loss: 15.5280
0.05
Epoch 4/10
161533/161533 [==============================] - 14s 89us/step - loss: 14.8446 - val_loss: 15.5280
0.05
Epoch 5/10
161533/161533 [==============================] - 14s 89us/step - loss: 14.8446 - val_loss: 15.5280
0.025
Epoch 6/10
161533/161533 [==============================] - 14s 89us/step - loss: 14.8446 - val_loss: 15.5280
0.025
Epoch 7/10
161533/161533 [==============================] - 15s 92us/step - loss: 14.8446 - val_loss: 15.5280
0.0125
Epoch 8/10
161533/161533 [==============================] - 16s 98us/step - loss: 14.8446 - val_loss: 15.5280
0.0125
Epoch 9/10
161533/161533 

**Using the embedding + train features**


Here we use both the original features in the data and the embedding we created to try to predict the loyalty value. there is no real improvment to the ANN model and it's seems the model is unable to learn as well as we would like.  

In [0]:
emb_mod = Model(input = nn.input, output = nn.get_layer('concatenate').output)
emb_out = emb_mod.predict([processed_train[f] for f in processed_train.columns])

max_date = train['first_active_month'].dt.date.max()

def date_parse(df):
  date_parts = ['year', 'month' , 'weekday']
  for part in date_parts:
    date_part_col = 'first_active_month_' + part
    df[date_part_col] = getattr(df['first_active_month'].dt, part).astype(int)
  df['elasped_time'] = (max_date - df['first_active_month'].dt.date).dt.days
  return df


new_train = date_parse(train).drop(['first_active_month','card_id','target'], axis=1)
emb_out = emb_out.reshape((emb_out.shape[0]), n_emb_output * 3)
pred =  pd.DataFrame(emb_out)
new_train = pd.concat([new_train, pred], axis=1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("co...)`
  """Entry point for launching an IPython kernel.


In [0]:
inp = Input(shape=(n_emb_output * 3 + 7,))
z = Dense(256, activation='relu', kernel_initializer='he_normal')(inp)
z = BatchNormalization()(z)
z = Dense(128, activation='relu', kernel_initializer='he_normal')(z)
z = BatchNormalization()(z)
z = Dense(64, activation='relu', kernel_initializer='he_normal' )(z)
z = BatchNormalization()(z)
z = Dense(1, activation='relu', kernel_initializer='he_normal')(z)

model = Model(inputs = inp, outputs= z)

sgd = SGD(lr=0.1, momentum=0.9, nesterov=True)

model.compile(loss = 'mse', optimizer = sgd)

In [0]:
history = model.fit(x=new_train, y=train.target, batch_size=64, epochs=5, validation_split = 0.2, callbacks=[reduce_lr, print_lr])

Train on 161533 samples, validate on 40384 samples
Epoch 1/5
161533/161533 [==============================] - 29s 182us/step - loss: 14.9101 - val_loss: 15.5280
0.1
Epoch 2/5
161533/161533 [==============================] - 28s 173us/step - loss: 14.8445 - val_loss: 15.5280
0.1
Epoch 3/5
161533/161533 [==============================] - 28s 173us/step - loss: 14.8445 - val_loss: 15.5280
0.05
Epoch 4/5
161533/161533 [==============================] - 28s 173us/step - loss: 14.8445 - val_loss: 15.5280
0.05
Epoch 5/5
161533/161533 [==============================] - 28s 172us/step - loss: 14.8445 - val_loss: 15.5280
0.025


**Using feature extraction for a novel ml algorithm from the ANN using the original features and embedding**

In this part we try to train a RandomForestRegressor model using original features of the train data and the feautres we extracted from our ANN model that used both the original features and the embeddings.

the extraction was performed like in previous work which means we took the output from the second to lsat layer in the ANN model that used both original features and embedded features and extract it as new 64 features we then used with the original features  as input for the random forest model.
the result were an improvemnet from the previous ANN models but not as significant as we would like.

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

ftr_extr_model = Model(input = model.input, output = model.layers[-2].output)

ftr_extr_in = ftr_extr_model.predict(new_train)

x_train, x_val, y_train, y_val = train_test_split(ftr_extr_in, train.target, test_size=0.2, random_state=42)

ml_model = RandomForestRegressor(n_estimators=150, criterion='mse', max_depth=7, 
                                 min_samples_leaf=15, max_features=6, 
                                 bootstrap=True, random_state=42)
ml_model.fit(x_train, y_train)


train_pred = ml_model.predict(x_train)
val_pred = ml_model.predict(x_val)

print('Train loss - {}'.format(np.sqrt(np.mean((train_pred - y_train)**2))))
print('Val loss - {}'.format(np.sqrt(np.mean((val_pred - y_val)**2))))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ba...)`
  after removing the cwd from sys.path.


Train loss - 3.8325267059143635
Val loss - 3.8676051399380498


Final remarks:
we hope we understood the concept behind categorical embeddings. at first we spend a lot of time and effort doing embedding in a way we then realized isnt correct and wasnt the point of the assignment which resulted in a lot of time wasted without real results we can show here.

in the end the embedding we crated did not improve the model at all so either we didnt understtod what embeddings meant here or that there is no real value for embedding on the elo dataset.

out best result was achive by using normal feature engineering and lgb model in the end with no connection to DL and/or embedding.

Also, the elo data is very uncelar and untill now we are not sure what actually we tried to predict in it and what would have been the best way to do it.